# Preprocesamiento de datos

Para evitar tener que preprocesar los datos cada vez que abrimos el Google Colab vamos a realizar tanto particionado de los datos como su preprocesamiento en este notebook aparte.

## Fase 1 - Cargar el dataset

Lo primero es cargar el dataset con Pandas y seleccionar la cantidad de filas que queremos usar:

In [1]:
import pandas as pd

raw_data = pd.read_csv("dataset_definitivo.csv")
print("general counts")
print(raw_data['personality'].value_counts())
training_data = raw_data[:10000]

print("used dataset info")
print(training_data.shape)
print(training_data.head())
print(training_data['personality'].value_counts())

general counts
personality
7     271107
3     227906
8     198143
4     183628
11    139942
12    128444
15     83261
5      65455
6      63511
2      58852
16     53388
1      46791
10     27698
9      19894
14     16627
13     14381
Name: count, dtype: int64
used dataset info
(10000, 2)
   personality                                               post
0            4  @Pericles216 @HierBeforeTheAC @Sachinettiyil T...
1            4  @HierBeforeTheAC @Pericles216 @Sachinettiyil A...
2            4  @HierBeforeTheAC @Pericles216 @Sachinettiyil Y...
3            4  @HierBeforeTheAC @Pericles216 @Sachinettiyil Y...
4            4  @HierBeforeTheAC @Pericles216 @Sachinettiyil T...
personality
4     4972
8     4316
16     712
Name: count, dtype: int64


## Fase 2 - Filtrado de Stopwords, Tokenización y Stemming

El preprocesamiento de texto se realiza usando las stopwords y el tokenizer de la librería de lenguaje natural de Python. Primero definimos las stopwords, que serán las de la librería de Python y las personalidades Myers-Briggs para evitar introducir sesgos, como que, si se menciona una personalidad, que esta no se tenga en cuenta para decidir la del autor. Luego se usa el PorterStemmer para obtener la “raíz” de las palabras. Entonces el resultado es una nueva columna que contiene los posts preprocesados.

In [2]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from tqdm.contrib.concurrent import process_map
from tqdm import tqdm
from multiprocessing import cpu_count
import re

import nltk
nltk.download('punkt')
nltk.download('stopwords')

ps = PorterStemmer()


STOPS = set(stopwords.words("english"))
CUSTOM_STOPS = set(['istj', 'isfj', 'infj', 'intj',
                    'istp', 'isfp', 'infp', 'intp',
                    'estp', 'esfp', 'enfp', 'entp',
                    'estj', 'esfj', 'enfj', 'entj'])

def process_text(post):
    try:
        post = post.lower()
        post = re.sub('https?://[^\s<>"]+|www\.[^\s<>"]+',' ',post) # filtrar links
        post = re.sub('[0-9]+',' ',post) # filtrar numeros
        post = re.sub('@[^\s]+',' ',post) # filtrar menciones de Twitter
        post = re.sub('[^0-9a-z]',' ',post) # filtrar caracteres especiais
        post = re.sub('[a-z]{1,2,3}', ' ', post) # filtrar monogramas, bigramas y trigramas
        return " ".join(
            [ps.stem(w) for w in word_tokenize(post)
             if not w in STOPS and w not in CUSTOM_STOPS])
    except:
        print("problem with: ", post)

preprocessedData = training_data.loc[:]

num_processes = cpu_count()

preprocessedData['processed_text'] = process_map(
                    process_text, training_data['post'],
                    max_workers=num_processes, chunksize=10)

preprocessedData

[nltk_data] Downloading package punkt to /home/mario/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/mario/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


  0%|          | 0/10000 [00:00<?, ?it/s]

,personality,post,processed_text
0,4,@Pericles216 @HierBeforeTheAC @Sachinettiyil T...,pope infal cathol dogma mean
1,4,@HierBeforeTheAC @Pericles216 @Sachinettiyil A...,perpetu entail church elect new po
2,4,@HierBeforeTheAC @Pericles216 @Sachinettiyil Y...,open door uniron nonsens believ nonsens
3,4,@HierBeforeTheAC @Pericles216 @Sachinettiyil Y...,know faith lol tri say perpetua
4,4,@HierBeforeTheAC @Pericles216 @Sachinettiyil T...,like say gon na give bike never b
...,...,...,...
9995,16,Welcome to poverty. https://t.co/exnhGEKM4F,welcom poverti
9996,16,Democrats love mail in ballots. https://t.co/t...,democrat love mail ballot
9997,16,I miss training to be a boxer. No one ever exp...,miss train boxer one ever expect kid see come ...
9998,16,I spend about 500$ a week on hotels. https://t...,spend week hotel


## Fase 3 - Particionar el dataset para entrenamiento y validación

Se divide el dataset en 3 partes para tener 60% como datos de entrenamiento, 20% validación y 20% pruebas.

In [3]:
from sklearn.model_selection import train_test_split

X = preprocessedData['processed_text']
Y = preprocessedData['personality']

X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.2, random_state=42)

X_train, X_val, y_train, y_val = train_test_split(
    X, Y, test_size=0.25, random_state=42)


## Fase 4 - Bolsa de palabras

Se crea una bolsa de palabras utilizando TfidVectorizer, que se basa en la frecuencia de las palabras para determinar su importancia.

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Datos de entrenamiento
bagOfWordsModel = TfidfVectorizer()
X_train = bagOfWordsModel.fit_transform(X_train)
print("X_train bag of words:")
print(X_train.shape)

# Datos de validación
X_val = bagOfWordsModel.transform(X_val)
print("X_val bag of words:")
print(X_val.shape)

# Datos pruebas
X_test = bagOfWordsModel.transform(X_test)
print("X_test bag of words:")
print(X_test.shape)

X_train bag of words:
(7500, 8470)
X_val bag of words:
(2500, 8470)
X_test bag of words:
(2000, 8470)


## Fase 5 - Reducción de dimensionalidad

Utilizamos TruncatedSVD para reducir la dimensionalidad de los datos de entrenamiento y prueba, permitiendo así trabajar con un número menor de características manteniendo la información relevante para el modelo. Esto puede ser útil en casos donde se tiene un alto número de características y se desea reducir la complejidad del modelo o mejorar los tiempos de entrenamiento.

Para ello usamos la técnica de reducción de dimensionalidad Truncated SVD, se suele usar para NLP por permitir trabajar con matrices dispersas y grandes cantidades de datos más rápido.

Este método tiene un problema denominado "sign indeterminacy", que explicado de forma sencilla, hace que los resultados entre sean inconsistentes usando los mismos parámetros y datos. Para evitarlo los desarrolladores recomiendan guardar la instancia inicial y usarla para procesar nuevas entradas.

In [5]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=1000, n_oversamples=15, random_state=42)
X_train = svd.fit_transform(X_train)
X_val = svd.transform(X_val)
X_test = svd.transform(X_test)

print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

(7500, 1000)
(2500, 1000)
(2000, 1000)


## Fase 6 - Guardar el resultado

Utilizamos joblib para guardar estas bolsas de palabras para poder usarlas para entrenamiento sin tener que repetir el proceso. Elegimos esta librería respecto a Pickle, que ya viene incluido en Python, porque permite comprimir el fichero resultante simplemente añadiendo una extensión de fichero comprimido al fichero de salida, así podemos enviar modelos preentrenados y datos de distintas etapas más rápido.

In [6]:
from tqdm.contrib.concurrent import process_map,thread_map
from joblib import dump

def multiargs_dump(args_tuple):
    dump(args_tuple[0], args_tuple[1])


thread_map(multiargs_dump,
    (
        (X_train, "X_train.lzma"),
        (X_test, "X_test.lzma"),
        (X_test, "X_val.lzma"),
        (y_train, "y_train.lzma"),
        (y_test, "y_test.lzma"),
        (y_test, "y_val.lzma"),
        (bagOfWordsModel, "bag_of_words.lzma"),
        (svd, "svd.lzma")
    ))


  0%|          | 0/8 [00:00<?, ?it/s]

[None, None, None, None, None, None, None, None]